In [66]:
import string
import os
import numpy as np
import tensorflow as tf

np.random.seed(0)

In [116]:
data_path = os.path.expanduser('~/Datasets/text8/text8.txt')
vocabulary_size = len(string.ascii_lowercase) + 1 # [a-z] + ' '
first_letter = ord(string.ascii_lowercase[0])

with open(data_path) as f:
  text = f.read()

def char2id(char):
  if char in string.ascii_lowercase:
    return ord(char) - first_letter + 1
  elif char == ' ':
    return 0
  else:
    print('Unexpected character: %s' % char)
    return 0
  
def id2char(dictid):
  if dictid > 0:
    return chr(dictid + first_letter - 1)
  elif dictid == 0:
    return ' '
  else:
    print('Unexpected codepint: %s' % dictid)
    return ' '

def make_batches(batch_size, num_unrollings):
  slice_len = len(text) // batch_size
  n_batches = slice_len // num_unrollings
  
  unrollings = []
  for i in range(slice_len):
    unrolling = np.zeros((batch_size, vocabulary_size))
    for j in range(batch_size):
      char = text[slice_len * j + i]
      unrolling[j, char2id(char)] = 1
      
    unrollings.append(unrolling)
    
  for i in range(n_batches):
    batch = unrollings[i * num_unrollings:(i + 1) * num_unrollings + 1]
    yield batch

In [112]:
batches = make_batches(32, 20) 
batch = next(batches) + next(batches)

assert(len(batch) == 21 + 21)

for row in range(3):
  for u in batch:
    assert u.shape == (32, 27)
    print(id2char(np.argmax(u[row])), end='')
  print('')
  
del batches

 anarchism originatedd as a term of abuse 
esident lyndon johnsoon signed a proclamat
 minting afonso also  sent ambassadors to 


In [206]:
class LSTM(object):
  def __init__(self, input_dim, output_dim):
    # Input gate: input, previous output, and bias.
    self.ix = tf.Variable(tf.truncated_normal([input_dim, output_dim], -0.1, 0.1))
    self.im = tf.Variable(tf.truncated_normal([output_dim, output_dim], -0.1, 0.1))
    self.ib = tf.Variable(tf.zeros([1, output_dim]))
    # Forget gate: input, previous output, and bias.
    self.fx = tf.Variable(tf.truncated_normal([input_dim, output_dim], -0.1, 0.1))
    self.fm = tf.Variable(tf.truncated_normal([output_dim, output_dim], -0.1, 0.1))
    self.fb = tf.Variable(tf.zeros([1, output_dim]))
    # Memory cell: input, state and bias.                             
    self.cx = tf.Variable(tf.truncated_normal([input_dim, output_dim], -0.1, 0.1))
    self.cm = tf.Variable(tf.truncated_normal([output_dim, output_dim], -0.1, 0.1))
    self.cb = tf.Variable(tf.zeros([1, output_dim]))
    # Output gate: input, previous output, and bias.
    self.ox = tf.Variable(tf.truncated_normal([input_dim, output_dim], -0.1, 0.1))
    self.om = tf.Variable(tf.truncated_normal([output_dim, output_dim], -0.1, 0.1))
    self.ob = tf.Variable(tf.zeros([1, output_dim]))
  
  # Definition of the cell computation.
  def __call__(self, i, o, state):
    """Create a LSTM cell. See e.g.: http://arxiv.org/pdf/1402.1128v1.pdf
    Note that in this formulation, we omit the various connections between the
    previous state and the gates."""
    input_gate = tf.sigmoid(tf.matmul(i, self.ix) + tf.matmul(o, self.im) + self.ib)
    forget_gate = tf.sigmoid(tf.matmul(i, self.fx) + tf.matmul(o, self.fm) + self.fb)
    update = tf.matmul(i, self.cx) + tf.matmul(o, self.cm) + self.cb
    state = forget_gate * state + input_gate * tf.tanh(update)
    output_gate = tf.sigmoid(tf.matmul(i, self.ox) + tf.matmul(o, self.om) + self.ob)
    return output_gate * tf.tanh(state), state

In [ ]:
num_unrollings = 20
batch_size = 32
n_h = 256

tf.reset_default_graph()

global_step = tf.Variable(0, name='global_step', trainable=False)
learning_rate = tf.placeholder(tf.float32, name='learning_rate')

train_data = list()
for i in range(num_unrollings + 1):
  train_data.append(tf.placeholder(tf.float32, shape=[batch_size, vocabulary_size], name='x_%s' % i))
  
xs = train_data[:num_unrollings]
ys = train_data[1:]  # labels are inputs shifted by one time step.

saved_output = tf.Variable(tf.zeros([batch_size, n_h]), trainable=False)
saved_state = tf.Variable(tf.zeros([batch_size, n_h]), trainable=False)
saved_output2 = tf.Variable(tf.zeros([batch_size, n_h]), trainable=False)
saved_state2 = tf.Variable(tf.zeros([batch_size, n_h]), trainable=False)
saved_output3 = tf.Variable(tf.zeros([batch_size, n_h]), trainable=False)
saved_state3 = tf.Variable(tf.zeros([batch_size, n_h]), trainable=False)

lstm = LSTM(input_dim=vocabulary_size, output_dim=n_h)
lstm2 = LSTM(input_dim=n_h, output_dim=n_h)
lstm3 = LSTM(input_dim=n_h, output_dim=n_h)
outputs = list()
output = saved_output
state = saved_state
output2 = saved_output2
state2 = saved_state2
output3 = saved_output3
state3 = saved_state3
for x in xs:
  output, state = lstm(x, output, state)
  output2, state2 = lstm2(output, output2, state2)
  output3, state3 = lstm3(output2, output3, state3)
  outputs.append(output3)
    
# classifier weights and biases.
w = tf.Variable(tf.truncated_normal([n_h, vocabulary_size], -0.1, 0.1))
b = tf.Variable(tf.zeros([vocabulary_size]))

with tf.control_dependencies([saved_output.assign(output),
                              saved_state.assign(state),
                              saved_output2.assign(output2),
                              saved_state2.assign(state2),
                              saved_output3.assign(output3),
                              saved_state3.assign(state3)]):
  y = tf.concat(ys, axis=0)
  z = tf.nn.xw_plus_b(tf.concat(outputs, axis=0), w, b)
  loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y, logits=z))
  correct_prediction = tf.equal(tf.argmax(tf.nn.softmax(z), axis=1), tf.argmax(y, axis=1))
  accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
  
train = tf.train.AdamOptimizer(learning_rate).minimize(loss, global_step)
# gradient clipping
# optimizer = tf.train.AdamOptimizer(learning_rate)
# gradients, v = zip(*optimizer.compute_gradients(loss))
# gradients, _ = tf.clip_by_global_norm(gradients, 1.25)
# train = optimizer.apply_gradients(
#   zip(gradients, v), global_step=global_step)

with tf.name_scope('summary'):
  tf.summary.scalar('loss', loss)
  tf.summary.scalar('accuracy', accuracy)
  
merged = tf.summary.merge_all()
init = tf.global_variables_initializer()

In [192]:
batches = make_batches(batch_size, num_unrollings)
batch = next(batches)

In [ ]:
fname = 'rnn'
log_dir = '/tmp/tf_log/%s' % fname
model_path = os.path.join(log_dir , 'model_2layer.ckpt')
restore = False
lr = 0.01
steps = 100000
log_interval = 200
save_interval = 1000

saver = tf.train.Saver()

with tf.Session() as sess:
  writer = tf.summary.FileWriter(log_dir, sess.graph)
  
  if restore:
    saver.restore(sess, model_path)
  else:
    sess.run(init)
    
  for i in range(sess.run(global_step), steps):
    batch = next(batches)
    feed_dict = {learning_rate: lr}
    for j, input in enumerate(train_data):
      feed_dict[input] = batch[j]
    
    sess.run(train, feed_dict)
      
    if i % log_interval == 0:
      l, a, summary = sess.run([loss, accuracy, merged], feed_dict)
      print('step: %s, loss: %s, accuracy: %s' % (i, l, a))
      writer.add_summary(summary, i)
      writer.flush()
      
    if i % save_interval == 0:
      save_path = saver.save(sess, model_path)
      print('model saved: %s' % save_path)
      
  wrtier.close()

step: 0, loss: 3.37448, accuracy: 0.126563
model saved: /tmp/tf_log/rnn/model_2layer.ckpt
